In [ ]:
!pip uninstall -y numpy pyarrow datasets spacy scikit-learn
!pip install numpy==1.26.4
!pip install pyarrow==14.0.2 datasets==2.14.6
!pip install spacy scikit-learn
!python -m spacy download en_core_web_sm

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: datasets 2.14.6
Uninstalling datasets-2.14.6:
  Successfully uninstalled datasets-2.14.6
Found existing installation: spacy 3.8.11
Uninstalling spacy-3.8.11:
  Successfully uninstalled spacy-3.8.11
Found existing installation: scikit-learn 1.8.0
Uninstalling scikit-learn-1.8.0:
  Successfully uninstalled scikit-learn-1.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 67.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, which is not installed.
yellowbrick 1.5 

  Using cached pyarrow-14.0.2-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
  Using cached datasets-2.14.6-py3-none-any.whl.metadata (19 kB)
Using cached pyarrow-14.0.2-cp312-cp312-manylinux_2_28_x86_64.whl (38.0 MB)
Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.31.0 requires pyarrow>=15.0.2, but you have pyarrow 14.0.2 which is incompatible.
  Using cached scikit_learn-1.8.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 41.7 MB/s eta 0:00:00
Using cached scikit_learn-1.8.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (8.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [ ]:
import numpy, pyarrow, datasets

print("NumPy:", numpy.__version__)
print("PyArrow:", pyarrow.__version__)
print("Datasets:", datasets.__version__)


NumPy: 1.26.4
PyArrow: 14.0.2
Datasets: 2.14.6


In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
import spacy
from spacy.tokens import Doc
from spacy.training import Example

nlp = spacy.blank("en")

def convert_to_spacy(dataset_split):
    examples = []
    for item in dataset_split:
        words = item["tokens"]
        labels = item["ner_tags"]

        text = " ".join(words)
        doc = nlp.make_doc(text)

        ents = []
        start = 0
        for word, label in zip(words, labels):
            end = start + len(word)
            if label != 0:  # 0 = O (outside)
                ent_label = dataset["train"].features["ner_tags"].feature.names[label]
                ent_label = ent_label.split("-")[-1]
                ents.append((start, end, ent_label))
            start = end + 1

        doc.ents = [doc.char_span(s, e, label=l) for s, e, l in ents if doc.char_span(s, e)]
        examples.append(doc)

    return examples

train_docs = convert_to_spacy(dataset["train"].select(range(1000)).to_list())
test_docs  = convert_to_spacy(dataset["test"].select(range(200)).to_list())

In [ ]:
from spacy.pipeline import EntityRecognizer

ner = nlp.add_pipe("ner")

# Add labels
for doc in train_docs:
    for ent in doc.ents:
        ner.add_label(ent.label_)

optimizer = nlp.initialize()

for epoch in range(10):
    losses = {}
    for doc in train_docs:
        example = Example.from_dict(doc, {"entities": [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]})
        nlp.update([example], drop=0.3, losses=losses)
    print(f"Epoch {epoch+1}, Loss: {losses['ner']}")

Epoch 1, Loss: 2475.9290042141697
Epoch 2, Loss: 1701.578626734503
Epoch 3, Loss: 1165.5600604374515
Epoch 4, Loss: 840.4947681227077
Epoch 5, Loss: 603.8529887622174
Epoch 6, Loss: 493.7397136472677
Epoch 7, Loss: 388.36115262917195
Epoch 8, Loss: 374.6727805856233
Epoch 9, Loss: 350.90083449344274
Epoch 10, Loss: 299.0159053851803


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

y_true = []
y_pred = []

for doc in test_docs:
    pred_doc = nlp(doc.text)

    true_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents}
    pred_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in pred_doc.ents}

    for ent in true_ents:
        y_true.append(ent[2])
        y_pred.append(ent[2] if ent in pred_ents else "O")

    for ent in pred_ents - true_ents:
        y_true.append("O")
        y_pred.append(ent[2])

labels = list(set(y_true) - {"O"})

precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=labels, average="weighted"
)

accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")

Accuracy : 0.6137
Precision: 0.7698
Recall   : 0.7420
F1-score : 0.7555


In [ ]:
text = "Elon Musk is the CEO of Tesla and SpaceX based in California."

doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Elon PER
Musk PER
CEO MISC
Tesla ORG
SpaceX LOC
California LOC
